In [1]:
import pandas

In [2]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [3]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 12724314714593405967, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 4945621811
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 9289714587106456863
 physical_device_desc: "device: 0, name: GeForce GTX 1060 6GB, pci bus id: 0000:01:00.0, compute capability: 6.1"]

In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("input\AllDogImages"))
# print(os.listdir("../input/images/images"))

# Any results you write to the current directory are saved as output.

['01-AllDogImages']


In [5]:
# from keras.applications.inception_v3 import InceptionV3
from keras.applications.xception import Xception
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

from collections import Counter
from scipy.ndimage import imread
from scipy.io import loadmat
from scipy.misc import imresize
import os
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
import numpy

Using TensorFlow backend.


In [6]:
base_model = Xception(weights='imagenet', include_top=False)

Instructions for updating:
Colocations handled automatically by placer.


In [7]:
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
# x = Dropout(0.5)(x)
x = Dense(100, activation='relu')(x)
# This layer being at 1024 isn't bad. ^
x = Dropout(0.5)(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(140, activation='softmax')(x)
# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [8]:
model.summary()

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=["acc"])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, None, None, 3 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, None, None, 3 0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [10]:
# load images
image_dir = "input/AllDogImages/01-AllDogImages"
basenames = [f for f in os.listdir(image_dir) if(f[-3:] == "jpg")][:]
targs = ['_'.join(f.split("_")[:-1]) for f in basenames]
image_paths = [os.path.join(image_dir, f) for f in basenames]

images = [imread(imp) for imp in image_paths]
grays = {i for i, im in enumerate(images) if(len(im.shape) != 3)}
images = [im for i, im in enumerate(images) if(i not in grays)]
targs = [t for i, t in enumerate(targs) if(i not in grays)]
images = [im if(im.shape[2]==3) else im[:,:,:-1] for im in images]
# resize all images
im_size = (200,200)
# try 224x224, cause that's what VGG uses and we're going to be swapping models
images = [imresize(im, size=im_size) for im in images]

#def data_gen():
#    while(True):
        #Choose N random images from training set
        # read those images
        # preprocess, convert cats to one-hot, etc
#        yield (images, labels)

d:\anaconda\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0.
Use ``matplotlib.pyplot.imread`` instead.
  import sys


UnboundLocalError: local variable 'photoshop' referenced before assignment